In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import openpyxl
import io
import numpy as np
from shutil import copyfile
import datetime
from datetime import date
from datetime import time
import time
import webbrowser
from googlesearch import search 
import numpy

year='2021'
year_last='2020'
year_last2='2019' 



# INITIALIZE PLAYER TABLE

In [2]:
#read in list of players/team/positions
player_df = pd.read_csv('NFL_player_list.csv')

#make blank lists
gamelog_list = []
roto_list = []

#loop through each player name 
for j in player_df['PLAYER']:
    
    #create url for each player for his pro-football-ref game log page
    player_ref = j + " pro-football-reference.com"
    for k in search(player_ref, tld="co.in", stop=1, pause=2):
        gamelog_list.append(k[0:-4] + "/gamelog/")
    
    #create each player's rotoworld blurb url
    player_roto = j + "rotoworld"
    for k in search(player_roto, tld="co.in", stop=1, pause=2):
        roto_list.append(k)

#add gamelog and rotoworld urls as columns to the player info table
player_df['Gamelog']=gamelog_list 
player_df['roto']=roto_list

#create excel tab name for each player which is team and name
player_df['excel-name']=player_df['TEAM'] + " " + player_df['PLAYER']

# exported updated table to new csv
player_df.to_csv('NFL_player_list_post.csv',index=False)
    

# Add new player sheets

In [3]:
# save of the last version of the NFL Prop Excel file
copyfile('NFL_Props_Latest.xlsx','NFL_Props_Previous.xlsx')

# read in player table
player_df = pd.read_csv('NFL_player_list_post2.csv')

# innitialize excel 
Prop_file = 'NFL_Props_Latest.xlsx'
book = openpyxl.load_workbook(Prop_file)
tab_list = book.sheetnames

# excel tempelates for each position type.
temp_sheet_qb=book["temp_qb"]
temp_sheet_rb=book["temp_rb"]
temp_sheet_wr=book['temp_wr']

# make new sheet for any player that doesn't already have a tab using the position tempelates and write rotoworld address to it

for j in range(0,player_df.shape[0]): # of rows
    try:
        if player_df['excel-name'][j] not in tab_list:
            if player_df['Pos'][j]=='QB':
                book.copy_worksheet(temp_sheet_qb).title = player_df['excel-name'][j] ## copy sheet with player name
                book[player_df['excel-name'][j]]['B9'] = player_df['roto'][j]
                book[player_df['excel-name'][j]]['B8'] = player_df['PLAYER'][j]
            elif player_df['Pos'][j]=='RB':
                book.copy_worksheet(temp_sheet_rb).title = player_df['excel-name'][j] ## copy sheet with player name
                book[player_df['excel-name'][j]]['B9'] = player_df['roto'][j]
                book[player_df['excel-name'][j]]['B8'] = player_df['PLAYER'][j]
            else: 
                book.copy_worksheet(temp_sheet_wr).title = player_df['excel-name'][j] ## copy sheet with player name
                book[player_df['excel-name'][j]]['B9'] = player_df['roto'][j]
                book[player_df['excel-name'][j]]['B8'] = player_df['PLAYER'][j]
    except:
        print('error ' + player_df['excel-name'][j]) 
            
book.save(Prop_file)       
        
## timestamp to keep track of each time this was run

secondsSinceEpoch = time.time()
# Convert seconds since epoch to struct_time
timeObj = time.localtime(secondsSinceEpoch)
print('Run TimeStamp is : %d-%d-%d %d:%d:%d' % (
timeObj.tm_mon, timeObj.tm_mday, timeObj.tm_year, timeObj.tm_hour, timeObj.tm_min, timeObj.tm_sec))


error Det Chase Daniel
Run TimeStamp is : 12-15-2020 22:7:11


import html table to panda: <br>
https://pandas.pydata.org/pandas-docs/stable/io.html#io-read-html

don't overwrite other tabs: <br>
https://stackoverflow.com/questions/20219254/how-to-write-to-an-existing-excel-file-without-overwriting-data-using-pandas

convert strings to numbers <br>
https://stackoverflow.com/questions/34844711/convert-entire-pandas-dataframe-to-integers-in-pandas-0-17-0/34844867#34844867

panda merge
https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

# UPDATE FILES

In [4]:
## open excel file
player_df = pd.read_csv('NFL_player_list_post.csv')
Prop_file = 'NFL_Props_Latest.xlsx'
book = openpyxl.load_workbook(Prop_file)
writer = pd.ExcelWriter(Prop_file,engine='openpyxl')
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

#create several blank dataframes
df_zeros = pd.DataFrame(numpy.zeros(shape=(1,10))) ## blank
df_zeros2 = pd.DataFrame(numpy.zeros(shape=(1,6))) ## blank
df_games= pd.DataFrame([10])

for j in range(0,player_df.shape[0]): #loop through every player in the csv
    
    ## reset df_log to blank dataframe
    df_log=pd.DataFrame()
    
    #### filter down by year you want
    ### sort by date
    ### add to excel
    try: 
        df_log=pd.read_html(player_df['Gamelog'][j], header=1, index_col=0 )[0]####0 for first table

        ## try playoffs

        try: 
            df_log_PO=pd.read_html(player_df['Gamelog'][j], header=1, index_col=0 )[1]####1 for playoff table
            df_log = pd.concat([df_log, df_log_PO],sort=False)  ## otherwise column get sorted alphabetically    ### combine table
        except:
            print('no playoffs for ' + player_df['excel-name'][j])


        ### if elif else for qb/rb/wr to rename columns and remove unneeded columns

        if player_df['Pos'][j]=='QB':
            df_log.drop(['G#'],axis=1,inplace=True)
            df_log.dropna(subset=['GS'],inplace=True)
            df_log.rename(columns={df_log.columns[5]:'at',df_log.columns[12]:'P Yds',df_log.columns[13]:'P TD',df_log.columns[21]:'Rush Yds',df_log.columns[20]:'Rush Att'}, inplace=True)
            df_log=df_log[['Week','Year','Date', 'Tm', 'at', 'Opp' ,'Result','GS', 'Cmp', 'Att','P Yds','P TD', 'Int', 'Rush Yds','Rush Att']]
        elif player_df['Pos'][j]=='RB':
            df_log.drop(['G#'],axis=1,inplace=True)
            if df_log.columns[9]=='Att':
                if df_log.columns[15]=='Yds.1':
                    df_log.rename(columns={df_log.columns[5]:'at',df_log.columns[15]:'Rec Yds',df_log.columns[17]:'Rec TD',df_log.columns[10]:'Rush Yds',df_log.columns[12]:'Rush TD'}, inplace=True)
                else:
                    df_log.rename(columns={df_log.columns[5]:'at',df_log.columns[10]:'Rush Yds',df_log.columns[12]:'Rush TD'}, inplace=True)
                    df_log["Rec"]=0
                    df_log["Rec Yds"]=0
                    df_log["Rec TD"]=0
            elif df_log.columns[16]=='Att':
                df_log.rename(columns={df_log.columns[5]:'at',df_log.columns[11]:'Rec Yds',df_log.columns[13]:'Rec TD',df_log.columns[17]:'Rush Yds',df_log.columns[19]:'Rush TD'}, inplace=True)
            else:
                df_log.rename(columns={df_log.columns[5]:'at',df_log.columns[11]:'Rec Yds',df_log.columns[13]:'Rec TD'}, inplace=True)
                df_log["Att"]=0
                df_log["Rush Yds"]=0
                df_log["Rush TD"]=0
            df_log=df_log[['Week','Year','Date', 'Tm', 'at', 'Opp' ,'Result','Pct','Att','Rush Yds','Rush TD','Rec', 'Rec Yds','Rec TD']]
        else: 
            df_log.drop(['G#'],axis=1,inplace=True)
            df_log.rename(columns={df_log.columns[5]:'at',df_log.columns[11]:'Rec Yds',df_log.columns[13]:'Rec TD'}, inplace=True)
            df_log=df_log[['Week','Year','Date', 'Tm', 'at', 'Opp' ,'Result','Pct','Rec','Rec Yds','Rec TD']]

        ### filter down to years you want    
        ####replace NaN with zeros


        df_log=df_log[df_log['Date'].str.contains(year_last)|df_log['Date'].str.contains(year)|df_log['Date'].str.contains(year_last2)].fillna(0).tail(22)

        ## sort by date order
        df_log.sort_values(by=['Date'],inplace=True)

        df_log['Date']=df_log['Year'].astype(int).astype(str)+' W'+df_log['Week'].astype(int).astype(str)
        df_log.drop(['Week','Year'],axis=1,inplace=True)

        ## write to excel
        df_log = df_log.apply(pd.to_numeric, downcast='float', errors='ignore')
        df_log.to_excel(writer,player_df['excel-name'][j],index=False,header=False,startcol=1,startrow=11,float_format="%.1f")
        df_zeros.to_excel(writer,player_df['excel-name'][j],index=False,header=False,startcol=7,startrow=9,float_format="%.1f")
        ## df_zeros2.to_excel(writer,player_df['excel-name'][j],index=False,header=False,startcol=7,startrow=8,float_format="%.1f")
        df_games.to_excel(writer,player_df['excel-name'][j],index=False,header=False,startcol=1,startrow=1,float_format="%.1f")

    except:
        print('## no gamelog for ' + player_df['excel-name'][j])
    
    
## close out of excel file    
writer.save()
book.save(Prop_file)   

## timestamp

secondsSinceEpoch = time.time()
# Convert seconds since epoch to struct_time
timeObj = time.localtime(secondsSinceEpoch)
print('Run TimeStamp is : %d-%d-%d %d:%d:%d' % (
timeObj.tm_mon, timeObj.tm_mday, timeObj.tm_year, timeObj.tm_hour, timeObj.tm_min, timeObj.tm_sec))


no playoffs for KC Clyde Edwards-Helaire
no playoffs for TB O.J. Howard
no playoffs for TB Justin Watson
Run TimeStamp is : 1-20-2021 21:27:42
